In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d9649d57-8901-4fc2-9acf-1a0fab7b81fd;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 81ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



<div style="color:lightgreen">
Eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

```
SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen
FROM dataFrame
WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?
HAVING COUNT(x) > 1
GROUP BY user_id;
```


Vorschlag Sebastian:
<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>
</div>

<div style="color:lightgreen">

Mein Vorschlag:

Ist es nicht besser direkt nach Nutzer und Zeit zu gruppieren? Dann hat man für jeden Nutzer direkt angezeigt, wie oft zu einem Zeitpunkt Pixel platziert wurden?
Dann wäre auch die Bereinigung der Daten effektiver als oben.

Hier der Versuch:
</div>

In [2]:
from src.features.feature_functions import group_dataframe_by_time_and_user
import pyspark.sql.functions as F

timeUserGrouped = group_dataframe_by_time_and_user(dataFrame)
#timeUserGrouped = group_dataframe_by_columns(dataFrame, ['t', 'user_id'])
timeUserGrouped.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [3]:
timeUserGrouped = timeUserGrouped.withColumnRenamed('count','time_count')

timeUserGrouped = timeUserGrouped.filter(timeUserGrouped.time_count > 1)
timeUserGrouped.show()

+--------------------+----+----------+--------------------+--------------------+-------------------------+
|             user_id|   t|time_count|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+--------------------+----+----------+--------------------+--------------------+-------------------------+
|X8gpDfO6+Z87SKKTy...|2974|         8|[1685, 904, 932, ...|[228, 1865, 1850,...|     [#FFD635, #000000...|
|AxBTmhBMbr1sbDw/2...|2344|         6|[1403, 777, 1741,...|[2, 8, 10, 2, 8, 10]|     [#FFA800, #FFA800...|
|DIECtVqoJqBlL6A0C...|1200|         6|[932, 920, 933, 9...|[1757, 1832, 1838...|     [#FFFFFF, #BE0039...|
|AxBTmhBMbr1sbDw/2...|1726|         6|[655, 612, 1575, ...|[1896, 1917, 7, 1...|     [#B44AC0, #E4ABFF...|
|FjHp8Fg+uuheibUWk...|3300|         4|[998, 998, 998, 998]|[1466, 1466, 1466...|     [#000000, #000000...|
|MsdvGyv1KSfWjE8OI...|1444|         4|[957, 954, 957, 954]|[1844, 1844, 1844...|     [#FFA800, #FFA800...|
|AxBTmhBMbr1sbDw/2...|3577|         4

In [8]:
grouped = timeUserGrouped.dropDuplicates(['user_id'])
grouped.show()

+--------------------+----+----------+---------------+---------------+-------------------------+
|             user_id|   t|time_count|collect_list(x)|collect_list(y)|collect_list(pixel_color)|
+--------------------+----+----------+---------------+---------------+-------------------------+
|++++KubvRsRI2sZI3...|1827|         2|   [1434, 1434]|     [360, 360]|       [#9C6926, #9C6926]|
|++0nCmngtZ68vbg6p...| 734|         2|     [757, 757]|     [592, 592]|       [#000000, #000000]|
|++3Ki0p8wbp0g7xQv...|3424|         2|   [1975, 1975]|       [92, 92]|       [#FFFFFF, #FFFFFF]|
|++3gcvcX6sgZwlgO7...|1574|         2|     [982, 982]|   [1528, 1528]|       [#000000, #000000]|
|++3l0k1HSXKiAsHkH...|1721|         2|     [664, 664]|   [1240, 1240]|       [#9C6926, #9C6926]|
|++4/asfR6j/RbsvIF...|2741|         2|     [467, 467]|     [937, 937]|       [#FFFFFF, #FFFFFF]|
|++4PiR2T1Y0vPxBnK...|3505|         2|     [205, 205]|   [1852, 1852]|       [#000000, #000000]|
|++5bQtHNJxwF1lYE2...| 349|   

In [9]:
print(grouped.count())

22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


733458


In [5]:
# holder = timeUserGrouped.drop('collect_list(x)').drop('collect_list(y)').drop('t').drop('time_count').drop('collect_list(pixel_color)')

# user = holder.take(2)
# print(user)
# a = user[1]
# print(a)
# b = a[9:]
# print(b)

<div style="color:lightgreen">
Mein Ziel damit war es, den Substring 'user_id=' aus dem Ergebnisstring zu entfernen und die Zeichen '

Damit ich das Ergebnis unten kontrollieren kann.
Hat aber nicht geklappt...
</div>

In [6]:

#user = timeUserGrouped.select('user_id').show(2)
#print(f'{user}')
#a = user[1]
#timeUserGrouped.where('user_id = a').show()

In [4]:
row = dataFrame.count('user_id')
print(f'Anzahl der herkömmlichen Zeilen: {row}')

row_sel = timeUserGrouped.count('user_id')
print(f'Anzahl der angeblichen Moderatoren: {row_sel}')

TypeError: count() takes 1 positional argument but 2 were given